<a href="https://colab.research.google.com/github/Jonchyk/Datamgmt/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

!pip install pyreadstat

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

In [158]:
#Background
#For PS1, I will be running three datasets - focused on early grade learning by region and language, pop change by region, and per capita income by region
#dataset 1: Foundational Learning Asessment -  an SPSS file found from UNICEF's Multi Cluster study in Kyrgystan, 2023 - https://mics.unicef.org/news/just-released-kyrgyzstan-2023-mics-survey-findings-snapshots-and-datasets
#will read this in, review, recode columns, and review the sample that I have to work with as well as make a plan of attack for creating scores by region and urban vs rural type
#The second dataset is population in millions in Kyrgyzstan. I'll run it, clean it, and calculate the percentage change per region year over year.
#purpose of this dataset is to understand demographic changes within Kyrgyzstan, leading up to the MCIS
#i'll also calculate a compound growth rate, to get a grasp on each region's change leading up to 2023.
#The Final Dataset is going to just look at the average income per region. Nothing fancy


In [159]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb' -O mics.sav


--2025-01-27 20:19:46--  https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb
Resolving docs.google.com (docs.google.com)... 142.251.2.139, 142.251.2.100, 142.251.2.113, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download [following]
--2025-01-27 20:19:46--  https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.137.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1807033 (1.7M) [application/octet-stream]
Saving to: ‘mics.sav’

mics.sav            100%[===================>]   1.72M  --.-KB/s    in 0.1s    

In [160]:
#read spss file
df = pd.read_spss("mics.sav")
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns', None)  # No limit on columns
#let's take a peek at this data set.
df.head(10)
#Let's specifically look at our geographic breakdown, since this is important
df["stratum"].value_counts()

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRE

,count
stratum,
Naryn - rural,480
Talas - rural,443
Osh city - urban,430
Osh - rural,387
Chui - rural,360
Batken - rural,327
Bishkek city - urban,308
Jala-Abad - rural,296
Issyk-kul - rural,284


In [161]:
#let's get a full list of columns
print(list(df.columns))

['HH1', 'HH2', 'LN', 'FS1', 'FS2', 'FS3', 'FSINT', 'FS4', 'FS5', 'FS6', 'FS7D', 'FS7M', 'FS7Y', 'FS9', 'FS10', 'FS17', 'FS8H', 'FS8M', 'FS11H', 'FS11M', 'FS12', 'FS13', 'FS14', 'FS15', 'FSHINT', 'FSFIN', 'FS15A', 'FS15C', 'FS15D', 'CB2M', 'CB2Y', 'CB3', 'CB4', 'CB5A', 'CB5B', 'CB6', 'CB7', 'CB8A', 'CB8B', 'CB9', 'CB10A', 'CB10B', 'CL1A', 'CL1B', 'CL1C', 'CL1X', 'CL3', 'CL4', 'CL5', 'CL6A', 'CL6B', 'CL6C', 'CL6D', 'CL6E', 'CL6X', 'CL7', 'CL8', 'CL9', 'CL10', 'CL11A', 'CL11B', 'CL11C', 'CL11D', 'CL11E', 'CL11F', 'CL11X', 'CL13', 'FCD2A', 'FCD2B', 'FCD2C', 'FCD2D', 'FCD2E', 'FCD2F', 'FCD2G', 'FCD2H', 'FCD2I', 'FCD2J', 'FCD2K', 'FCD3', 'FCD4', 'FCD5', 'FCF1', 'FCF2', 'FCF3', 'FCF6', 'FCF8', 'FCF10', 'FCF11', 'FCF12', 'FCF13', 'FCF14', 'FCF15', 'FCF16', 'FCF17', 'FCF18', 'FCF19', 'FCF20', 'FCF21', 'FCF22', 'FCF23', 'FCF24', 'FCF25', 'FCF26', 'PR3', 'PR5', 'PR6', 'PR7', 'PR8', 'PR9A', 'PR9B', 'PR10', 'PR11A', 'PR11B', 'PR12A', 'PR12B', 'PR12C', 'PR12X', 'PR13', 'PR15', 'FL1', 'FL2H', 'FL2M',

In [162]:
# Convert FL20A and FL20B to numeric if they are Categorical
df['FL20A'] = pd.to_numeric(df['FL20A'], errors='coerce')
df['FL20B'] = pd.to_numeric(df['FL20B'], errors='coerce')# Calculate the percentage of correct words
df['wrdprctcorrect'] = ((df['FL20A'] - df['FL20B']) / df['FL20A']) * 100

# Adjust values for cases where FL20B is 0 (all correct attempts)
df.loc[df['FL20B'] == 0, 'wrdprctcorrect'] = 100
df

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum,wrdprctcorrect
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE

In [163]:
#ok. Now that i've done this. I've identified all of the "columns" that need to be kept. it's a lot. Thankfully, we are keeping all of the FL codes (foundational learning)
#Let's drop all non necessary columns here, keeping language/geographic/ethnicity, grade, gender, age, questions on child labor, natural disasters, and the full foundational assessment questionaire
df = df.loc[:, df.columns.isin(['FS10', 'FS12', 'FS13', 'FS14', 'CB3', 'CB5B' , 'wrdprctcorrect', 'FL3' , 'FL7', 'FL9', 'FL20A', 'FL20B','FL21A', 'FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE', 'FL21D', 'FL21H', 'FL21I', 'FL21J', 'FL21K', 'FL21L',  'FL25C', 'FL25D', 'FL25E', 'FL26', 'FL26A', 'FL26B', 'FL26C', 'FL26D', 'FL26E', 'FL27A', 'FL27B', 'FL27C', 'FL27D', 'FL27E', 'FL28', 'ST1LANG', 'ST1WNUM', 'ST2LANG', 'ST2WNUM', 'HH4', 'HH6', 'HH7', 'HH52', 'HL4', 'ED5A', 'ED5B', 'ethnicity','wscore', 'windex5', 'windex10', 'stratum'])]

In [164]:
#let's get this down to 7 to 10 years old, since the data collected is for ages 7-14, but is a G2 foundational test. Wow!
df_age = df[(df["CB3"] > 6) & (df["CB3"] < 11)]
#how many respondents do we have now?
df_age["stratum"].value_counts()

,count
stratum,
Naryn - rural,152
Osh city - urban,143
Osh - rural,138
Talas - rural,132
Chui - rural,128
Batken - rural,106
Issyk-kul - rural,100
Bishkek city - urban,98
Jala-Abad - rural,97


In [165]:
#further, let's see how many of these children consented to taking the test.
df_age["FL3"].value_counts()

,count
FL3,
YES,1219
NO / NOT ASKED,48


In [166]:
#1219 said yes! Of those 1219, how many do we have in each "stratum" now?
df_age[["stratum", "FL3"]].value_counts(normalize=False)
#ok! This looks like it's enough data to work with.  While some of the urban smaples are a bit low, every sample per region is >30!


,,count
stratum,FL3,
Naryn - rural,YES,149
Osh city - urban,YES,130
Talas - rural,YES,128
Chui - rural,YES,124
Osh - rural,YES,114
Batken - rural,YES,100
Issyk-kul - rural,YES,100
Bishkek city - urban,YES,80
Jala-Abad - rural,YES,78


In [167]:
#run the dataframe and double check that we have only the columns listed above and all looks ok.
df_age

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21H,FL21I,FL21J,FL21K,FL21L,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,wscore,windex5,windex10,stratum,wrdprctcorrect
2,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 1,YES,71.0,4.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 4,Kyrgyz,0.390934,Fourth,8th decile,Issyk-kul - urban,94.366197
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,71.0,2.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,2.0,FEMALE,HIGHER,GRADE 5,Other ethnicity,1.751447,Richest,10th decile,Issyk-kul - urban,97.183099
17,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,71.0,5.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),INCORRECT,KYRGYZ,YES,CORRECT(2),NaN,CORRECT(ANNE),NaN,CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,RUSSIAN,71.0,KYRGYZ,47.0,30.0,URBAN,Issyk-kul,1.0,FEMALE,COMPLETE SECONDARY,GRADE 11,Kyrgyz,0.442114,Fourth,8th decile,Issyk-kul - urban,92.957746
18,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,59.0,5.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),INCORRECT,DOES NOT WANT TO TRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,3.0,FEMALE,HIGHER,GRADE 5,Kyrgyz,0.673772,Richest,9th decile,Issyk-kul - urban,91.525424
19,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 3,YES,59.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,30.0,URBAN,Issyk-kul,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 1,Kyrgyz,0.812186,Richest,9th decile,Issyk-kul - urban,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3924,YES,KYRGYZ,KYRGYZ,KYRGYZ,9.0,GRADE 4,YES,59.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRECT(NEAR A KARAGACH TREE),CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,NaN,NaN,NaN,NaN,NaN,NaN,CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,KYRGYZ,59.0,WAS NOT NECESSARY TO OFFER,NaN,90.0,URBAN,Osh c,4.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,1.101478,Richest,9th decile,Osh city - urban,100.000000
3925,YES,RUSSIAN,KYRGYZ,KYRGYZ,10.0,GRADE 5,YES,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),INCORRECT,CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOW...,DOES NOT WANT TO TRY,NaN,NaN,NaN,NaN,NaN,CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),INCORRECT,COMPLETED,RUSSIAN,71.0,WAS NOT N

In [168]:
#what are the counts for each grade.
df_age["CB5B"].value_counts( normalize=False)

,count
CB5B,
GRADE 2,349
GRADE 3,331
GRADE 4,304
GRADE 1,217
GRADE 5,111
GRADE 6,7
GRADE 10,0
GRADE 11,0
GRADE 7,0


In [169]:
#I want to filter out everyone that isn't in Grade 2 or Grade 3, since this is a grade 2 foundational test
filtered_df = df_age[df_age["CB5B"].isin(["GRADE 2", "GRADE 3"])]
#let's run this and see the count by stratum. Is this enough to be representative? Maybe ok if done at regional (non urban/rural level)
filtered_df["stratum"].value_counts( normalize=False)


,count
stratum,
Osh - rural,82
Osh city - urban,73
Naryn - rural,67
Talas - rural,66
Chui - rural,63
Batken - rural,53
Jala-Abad - rural,52
Issyk-kul - rural,49
Bishkek city - urban,47


In [170]:
filtered_df["FL20B"]

,FL20B
8,2.0
19,0.0
29,0.0
46,NaN
47,0.0
...,...
3920,NaN
3921,0.0
3923,0.0
3929,NaN


In [171]:
#There are two stories. The first story is read in a language selected by the child. If the child agrees to read a second story, in FL21D, they must pick a second language to read it in if they consent. Let's see that break out
pd.crosstab(df_age['FL21D'], df_age['ethnicity'])


ethnicity,Kyrgyz,Other ethnicity,Russian,Uzbek
FL21D,,,,
DOES NOT WANT TO TRY,228,7,10,47
KYRGYZ,89,5,2,13
NO RESPONSE,31,1,0,1
RUSSIAN,138,1,3,8
UZBEK,2,0,0,19


In [172]:
#Another question remains; the questionaire asks children what language they speak at home, versus hear at school (FL7, FL9). How does those counts come about?
pd.crosstab(df_age['FL7'], df_age['FL9'])
#by basis of ethnicity compared to language of instruction
pd.crosstab(df_age['ethnicity'], df_age['FL9'])
#by basis of ethnicity compared to language at home
pd.crosstab(df_age['ethnicity'], df_age['FL7'])



KeyError: 'FL7'

In [ ]:
#Let's rename certain columns, not all of them, but a few critical pieces - still working on this
df_age.rename(columns={'FS10': 'consent', 'FS17': 'completed', 'FS12': 'language_questionaire', 'FS13': 'language_interview', 'FS14': 'native_language', 'CB5': 'highest_grade'}, inplace=False)

In [ ]:
#check columns rename went through, use false first, then rerun below with true
df_age.head(1)

In [ ]:
#repeat df rename function but htis time with inplace = true

In [ ]:
#NEXT STEPS for this Dataset:
#ok, this is interesting. I'll need to look into this deeper when thinking about how to compare and contrast. Did students read in their language of instruction, or language at home?
#For the next PS, will's double check and create a column that returns 'languageathome1" or "languageofinstruction1", if the child chose to read a storyu, was it based on language at home or language of instruction?
#Will likely drop the second story. We'll assess foundational knowledge based on percentage of reading and math.
#Then run the average score in each of those  categories, based on the stratum (Oblast, rural/urban)
#then compare results by age, and by gender


In [ ]:
# In the future can do a merge and roll up, for now, let's keep as is. Need to export this to a CSV file, right?

In [ ]:
#now to read in a second dataset - let's grab some data from UNGLOBAL Data lab by region, it's quite clean and ready to go!
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8/export?format=csv' -O GDL-pop-data.csv

pop = pd.read_csv('/content/GDL-pop-data.csv')
pop.head(1)

In [ ]:
pop.drop(columns=['Country', 'Continent', 'ISOCode', 'Level', 'GDLCODE'], inplace=True)
pop.head(10)

In [ ]:
# Convert the relevant columns (excluding the first column 'Region') to numeric
for col in pop.columns[1:]:  # Skip the first column (Region)
    pop[col] = pd.to_numeric(pop[col], errors='coerce')  # Replace non-numeric values with NaN

# Now calculate the percentage change across columns (years), skipping NaN values
pop_pct_change = pop.drop('Region', axis=1).pct_change(axis='columns', fill_method=None) * 100

# Reattach the 'Region' column at the start (add it as the first column)
pop_pct_change.insert(0, 'Region', pop['Region'])

# Display the first 10 rows of the percentage change DataFrame with 'Region' column at the start
pop_pct_change.head(10)

In [ ]:
#Drop the NAN values
pop_pct_change.dropna(axis=1, how='any', inplace=True)

# Display the first 10 rows of the percentage change DataFrame
pop_pct_change.round(2)


In [ ]:
# Drop the NAN values
pop.dropna(axis=1, how='any', inplace=True)

# Function to calculate CAGR for each region (row)
def calculate_cagr(row):
    # Skip the first column (Region)
    years = len(row) - 1  # Exclude the Region column from the count
    start_value = row[1]  # Start value (first year data)
    end_value = row[years]  # End value (last year data)

    # If start_value or end_value is 0, return NaN
    if start_value == 0 or end_value == 0:
        return np.nan

    # Calculate CAGR
    try:
        cagr = ((end_value / start_value) ** (1 / years)) - 1
    except ZeroDivisionError:
        return np.nan  # Handle potential divide-by-zero error

    return round(cagr * 100, 2)  # Convert to percentage and round to two decimal places

# Apply the CAGR function to each row of the population DataFrame, excluding the Region column
pop['CAGR'] = pop.iloc[:, 1:].apply(calculate_cagr, axis=1)

# Display the updated DataFrame with the CAGR values
pop.head(10)

In [ ]:
#need to convert the dataframe csv if I want to download this
pop.to_csv('pop.csv', index=False)

#let's download this to csv for data visualization PS0 or PS1!
files.download('pop.csv')

In [ ]:
#Recap; we see that Bishkek had a CAGR throught he period 2003-2021 of 3.53%. This is more than double the total and any other region.
#that is to say, while the percentages look small, Bishkek has far exceeded the actual percentage growth. the nominal numbers above are in millions.
#So Bishkek doubled in size during this time period. most other places barely saw increases. Talk about everyone being packed into one small space!

In [ ]:
#Ok. Now need to convert a raw JSON file with per capita income monthly by region. I don't understand why downloading it led to a weird text file... Chatgpt lead the way!


# let's load the third data set - this was downloaded from Kyrgyzstan bureau of stats
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i' -O percapita.json
#Read in the raw JSON data directly from file
income = pd.read_json('/content/percapita.json')

# Check the structure of the loaded data to see how it is organized
print(income.head())

# Initialize a list to hold the rows of the DataFrame
rows = []

# Assuming the structure of 'income' is similar to your original JSON structure
# where 'data' is the key containing the region data
for region in income['data']:
    region_name = region['title_en']  # Get the region name
    values = region['values']  # Get the list of values for this region

    # Create a dictionary with region as the first entry and year-value pairs following
    region_data = {'Region': region_name}
    for year_data in values:
        year = year_data['key']
        value = year_data['value']
        region_data[year] = value

    # Append the row to the list
    rows.append(region_data)

# Create a DataFrame from the rows
income_df = pd.DataFrame(rows)
# Reorder the columns so that years are in ascending order (from 2019 to 2023, etc.)
year_columns = sorted([col for col in income_df.columns if isinstance(col, int)])  # Get only the year columns
ordered_columns = ['Region'] + year_columns  # Place 'Region' as the first column, followed by years

# Reorder the DataFrame columns
income_df = income_df[ordered_columns]

# Display the DataFrame
income_df

In [ ]:
# Define the exchange rate
exchange_rate = 87.45

# Convert the whole DataFrame from KGS to USD
income_df_usd = income_df.copy()

# Apply the conversion for each year (for all regions)
for year in income_df.columns[1:]:  # Skip the 'Region' column
    income_df_usd[year] = income_df[year] / exchange_rate

# Display the converted DataFrame
income_df_usd.round(2).head(10)

In [ ]:
#Code to calculate statistics for one year (2023) - monthly salary in USD per capita
# create "selection" for 2023
year_2023 = income_df_usd[2023]

# Calculate statistics for the selected year, defined as below
statistics_2023 = {
    'mean': year_2023.mean(),
    'median': year_2023.median(),
    'min': year_2023.min(),
    'max': year_2023.max()
}

statistics_2023
#yikes! per capita income is super low, ranging from 86 to 123 bucks.
#interesting to note that from a quick  search, it looks like the average wage is closer to double this
#Either this means that 1) ministry data is poor 2) there's a large non working part of the population between youth and adults
#(40% of nation under <18)that are captured in this metric, with about 10% above 60. this could be a good thing to dig further into!


In [ ]:
from google.colab import files
#convert dataframe to csv
income_df_usd.to_csv('income_usd.csv', index=False)

# Download the file, so I can use this for visual!
files.download('income_usd.csv')
